# Prediction of  'Credit_card', 'Savings', 'Morgage' in File2.zip

In [ ]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" 
value="Click here to toggle on/off the raw code."></form>''')

In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [3]:
fn = r'File2.zip'
df = pd.read_csv(fn, sep='|', encoding='cp1252')
df.columns
df.shape

# date_issued_bank
d = df['date_issued_bank'].astype(int).astype(str)
df['date_issued_bank'] = pd.to_datetime(d)

# date_birth
d = df['date_birth'].astype(int).astype(str)
df['date_birth'] = pd.to_datetime(d)

FileNotFoundError: [Errno 2] No such file or directory: 'File2.zip'

In [ ]:
# fix columns types
df['NUMCLI'] = df['NUMCLI'].astype(np.int64)
df['TOP_SALARIE'] = df['TOP_SALARIE'].astype(np.uint8)
df['AGE'] = df['AGE'].astype(np.uint8)
df['REVENU'] = df['REVENU'].astype(np.uint32)

df['Credit_card'] = df['Credit_card'].astype(np.uint8)
df['Savings'] = df['Savings'].astype(np.uint8)
df['Morgage'] = df['Morgage'].astype(np.uint8)

In [ ]:
df.head(8)

## Input data

**X** - vector of available features about clients :
* 'AGE' - age of customer in years (numeric)
* 'NUMCLI' - (numeric)
* 'REVENU' - (numeric)
* 'CodPostal' - (category)
* 'NB_child' - (numeric)
* 'Nationality' - (category)
* 'date_issued_bank' - skipped
* 'date_birth' - skipped
* 'Gender' - (category)
* 'Marital_status' - (category)
* 'Typ_residency' - (category)
* 'TOP_SALARIE' - (category)

## Prediction
**y** - values we want to predict 0/1: 'Credit_card', 'Savings', 'Morgage'.

In [ ]:
df.columns

In [ ]:
y = ['Credit_card', 'Savings', 'Morgage']

The predicted values are imbalanced (roughly 93% of 0s, 7% of 1s)

In [ ]:
bins=26
last_date = df['date_birth'].max()
df['age_days'] = (last_date - df['date_birth']).dt.days
df['age_days'].isna().sum() # no nulls, ok
df['age_days_qcut'] = pd.qcut(df['age_days'], bins, labels=False)
df['age_days_qcut'].value_counts()

In [ ]:
bins=12
df['REVENU'].dtype
df['REVENU'] = df['REVENU'].astype(np.uint32)
df['REVENU'].isna().sum() # no nulls, ok
df['REVENU_qcut'] = pd.qcut(df['REVENU'], bins, labels=False)
df['REVENU_qcut'].value_counts()

In [ ]:
df['NB_child'].isna().sum()  # some nulls
df['NB_child'].mean() # 0.85
df['NB_child'].max() # 9

df['NB_child'].fillna(1, inplace=True)
df['NB_child'] = df['NB_child'].clip(upper=4)
df['NB_child'] = df['NB_child'].astype(np.int8)
df['NB_child'].value_counts()

In [ ]:
bins=26
df['seniority_days'] = (df['date_issued_bank'].max() - df['date_issued_bank']).dt.days
df['seniority_days'].isna().sum() # no nulls, ok

q01 = df['seniority_days'].quantile(0.01)
q99 = df['seniority_days'].quantile(0.99)
df['seniority_days'] = df['seniority_days'].clip(q01, q99)

m = df['seniority_days']<q99
df['seniority_days_qcut'] = pd.qcut(df.loc[m, 'seniority_days'], bins, labels=False)
df.loc[-m , 'seniority_days_qcut'] = df['seniority_days_qcut'].max()+1 # last bin

df['seniority_days_qcut'].value_counts()

In [ ]:
df.loc[df['Nationality'].isna(), ['Nationality']] = 'unk'
df.loc[df['Nationality']=='D', ['Nationality']] = 'oth'
df.loc[df['Nationality']=='C', ['Nationality']] = 'oth'
df.loc[df['Nationality']=='0', ['Nationality']] = 'oth'
df['Nationality'].value_counts()

In [ ]:
# prepare X vector

# CodPostal, 10 most common codes, all remaining into single category
len(df['CodPostal'].value_counts())
sum(df['CodPostal'].isna())
df['CodPostal'].astype(str)
df['CodPostal'].max()
df['CodPostal'].min()

fq = df['CodPostal'].value_counts()
fq.head(10).index
m = -df['CodPostal'].isin(fq.head(10).index)
df.loc[m, 'CodPostal'] = 0
#df['CodPostal'] = df['CodPostal'].astype('category').cat.codes.astype(str)
df['CodPostal'] = df['CodPostal'].astype(int).astype(str)
df['CodPostal'].value_counts()
pd.get_dummies(df['CodPostal'])

# Gender, category
d = df['Gender'].value_counts()
# MR=mr.Monsieur MME=mrs.Madame MLE=miss.Mademoiselle
df['Gender'] = df['Gender'].map({'MR':'M', 'MME':'F', 'MLE':'F'})

# Marital_status, category
df['Marital_status'].value_counts()
df['Marital_status']

# Typ_residency
df['Typ_residency'].value_counts()

# TOP_SALARIE; 0,1
df['TOP_SALARIE'].value_counts()

In [ ]:
cat_names = ['age_days_qcut', 'REVENU_qcut', 'seniority_days_qcut',
             'NB_child', 'TOP_SALARIE',
             'Nationality', 'Gender', 'CodPostal', 'Marital_status', 'Typ_residency']
dfcat = pd.get_dummies(df[cat_names].astype(str), drop_first=True)

dfn = df[[]].copy()

dfc = pd.concat([dfn, dfcat], axis=1)

dfc.columns

In [ ]:
dfc

CodPostal I take 10 most common values, remaining are changed to 11th category other.

NB_child some values are missing, I fill in with mean values.

Gender will be only two values: Male, Female

All category variables are changed to dummy variables i.e.: 'Nationality', 'Gender', 'CodPostal', 'Marital_status', 'Typ_residency'

I add days = date_issued_bank - date_birth

All numerical variable will be scaled for use in log-regr.

In [ ]:
#%% add scaling
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler

#dfn_sc = dfn.copy()
#scale(dfn)
#scaler = StandardScaler()
#scaler.fit(dfn)
#dfn_sc[dfn.columns] = scaler.transform(dfn)

#dfc = pd.concat([dfn_sc, dfcat], axis=1)
dfc

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
X=dfc.copy()
X['_const']=1
vif = pd.DataFrame()
vif['VIF'] = [variance_inflation_factor(X.values, i) 
              for i in range(len(X.columns))]
vif['variable'] = X.columns
vif.iloc[:-1]

VIF if colinearity test. Log-reg should not get colinear variables as input. This does not harm predictions too much. This only makes model unstable (model parameters may become large) and prevents model parameters from having simple interpretation (as odds of events).

After I calculate VIF for the varibale set I remove those with VIF>5 (suspected colinearity). Removed: 'Typ_residency_En accession à la propriété', 'Typ_residency_Locataire', 'Typ_residency_Proprietaire'.

In [ ]:
#%% based on VIF I remove linearily correlated columns
dfc.drop(['Typ_residency_En accession à la propriété',
          'Typ_residency_Locataire',
          'Typ_residency_Proprietaire',
         ], axis=1, inplace=True)
dfc.columns

# Logistic regression classifier
## Predicion for 'Credit_card'

I will train my model by using only 80% of available data and I will use rest for testing (evaluating).

In [ ]:
#%% split to train and test
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X = dfc
y_names = 'Credit_card', 'Savings', 'Morgage'
y = df[y_names[0]]
y.value_counts()
y[y==0].size/y.size

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

I split into train (80%) and test (20%) sets. For now I use y='Credit_card'.

I apply LogisticRegression clasifier with regularization.

In [ ]:
#%% log regression
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred)*100:.2f}%')

Input set is imbalanced. This makes accuracy and confusion matrix not very informative.

In [ ]:
#%% confusion_matrix plot
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix
plot_confusion_matrix(model, X_test, y_test,
                      display_labels=('No', 'Yes'), cmap=plt.cm.Blues)
mat = confusion_matrix(y_test, y_pred)
mat

In [ ]:
#%% ROC
fpr, tpr, thres = roc_curve(y_test, y_pred_proba[:,1])

df_roc = pd.DataFrame(dict(fpr=fpr, tpr=tpr, fpr_=fpr))
ax = df_roc.plot.line(x='fpr', y='tpr', title=f'auc={auc(fpr, tpr):.4f}')
df_roc.plot.line(x='fpr', y='fpr_', ax=ax, grid=True, style='--')
ax.set_xlabel("False positive rate")
ax.set_ylabel("True positive rate")
ax.get_legend().remove()

By offering a Credit_card to 20% of most promising clients, we can expect to reach over 40% of total number of clients interested in Credit_card.

In [ ]:
X_test_full = X_test.copy()
X_test_full['y_pred_proba'] = y_pred_proba[:,1]
X_test_full['y_pred'] = y_pred
X_test_full['y_test'] = y_test
X_test_full.sort_values('y_pred_proba', ascending=False, inplace=True)
df_Credit_card = X_test_full.iloc[:20000]

df_Credit_card['y_test'].cumsum().reset_index(drop=True).plot(grid=True)

df_Credit_card = df_Credit_card.copy()
df_Credit_card['y_pred']=1
mat = confusion_matrix(df_Credit_card['y_test'], df_Credit_card['y_pred'])
mat

By offering a Credit_card to 20000 of most promising clients, we can expect to make 3106 sales. The total sale potential for all 100000 clients is 7325 sales, so we manage to hold over 40% of business by only addressing 20% of clients.

I will make one more check. I will display log-reg coeffitients for each variable. I observe that Nationality_F, CodPostal_69100 and Marital_status_Inconnu have P-value above 0.05 meaning, that they are not sufficiently informative for this model (and could be removed to simplify model).

I also observe that AGE, NB_child and Nationality_X have negative correlation with Credit_card. 


REVENU, TOP_SALARIE, Nationality_D, CodPostal_13127, Marital_status_Divorcé, Marital_status_Veuf, Typ_residency_Inconnu, Typ_residency_Proprietaire have positive correlation with Credit_card.

In [ ]:
import statsmodels.api as sm
X_train_ = sm.add_constant(X_train)
logit_model=sm.Logit(y_train, X_train_)
result=logit_model.fit()
print(result.summary2())

## Predicion for 'Savings'

In [ ]:
X = dfc
y_names = 'Credit_card', 'Savings', 'Morgage'
y = df[y_names[1]]
y.value_counts()
y[y==0].size/y.size
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred)*100:.2f}%')

In [ ]:
#%% ROC
fpr, tpr, thres = roc_curve(y_test, y_pred_proba[:,1])

df_roc = pd.DataFrame(dict(fpr=fpr, tpr=tpr, fpr_=fpr))
ax = df_roc.plot.line(x='fpr', y='tpr', title=f'auc={auc(fpr, tpr):.4f}')
df_roc.plot.line(x='fpr', y='fpr_', ax=ax, grid=True, style='--')
ax.set_xlabel("False positive rate")
ax.set_ylabel("True positive rate")
ax.get_legend().remove()

In [ ]:
X_test_full = X_test.copy()
X_test_full['y_pred_proba'] = y_pred_proba[:,1]
X_test_full['y_pred'] = y_pred
X_test_full['y_test'] = y_test
X_test_full.sort_values('y_pred_proba', ascending=False, inplace=True)
df_Credit_card = X_test_full.iloc[:20000]

df_Credit_card['y_test'].cumsum().reset_index(drop=True).plot(grid=True)

df_Credit_card = df_Credit_card.copy()
df_Credit_card['y_pred']=1
mat = confusion_matrix(df_Credit_card['y_test'], df_Credit_card['y_pred'])
mat, X_test_full['y_test'].sum()

By offering a Credit_card to 20000 of most promising clients, we can expect to make 1831 sales. The total sale potential for all 100000 clients is 6660 sales. We only retain 27% of business by restricting to 20% of all clients.

This is less helpful than with Credit_card.

## Predicion for 'Morgage'

In [ ]:
X = dfc
y_names = 'Credit_card', 'Savings', 'Morgage'
y = df[y_names[2]]
y.value_counts()
y[y==0].size/y.size
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred)*100:.2f}%')

In [ ]:
#%% ROC
fpr, tpr, thres = roc_curve(y_test, y_pred_proba[:,1])

df_roc = pd.DataFrame(dict(fpr=fpr, tpr=tpr, fpr_=fpr))
ax = df_roc.plot.line(x='fpr', y='tpr', title=f'auc={auc(fpr, tpr):.4f}')
df_roc.plot.line(x='fpr', y='fpr_', ax=ax, grid=True, style='--')
ax.set_xlabel("False positive rate")
ax.set_ylabel("True positive rate")
ax.get_legend().remove()

In [ ]:
X_test_full = X_test.copy()
X_test_full['y_pred_proba'] = y_pred_proba[:,1]
X_test_full['y_pred'] = y_pred
X_test_full['y_test'] = y_test
X_test_full.sort_values('y_pred_proba', ascending=False, inplace=True)
df_Credit_card = X_test_full.iloc[:20000]

df_Credit_card['y_test'].cumsum().reset_index(drop=True).plot(grid=True)

df_Credit_card = df_Credit_card.copy()
df_Credit_card['y_pred']=1
mat = confusion_matrix(df_Credit_card['y_test'], df_Credit_card['y_pred'])
mat, X_test_full['y_test'].sum()

By offering a Credit_card to 20000 of most promising clients, we can expect to make 2455 sales. The total sale potential for all 100000 clients is 6010 sales. We only retain 39% of business by restricting to 20% of all clients.

This is less helpful than with Credit_card.

# RandomForestClassifier

I will use all varaibles, including  'Typ_residency_En accession à la propriété', 'Typ_residency_Locataire', 'Typ_residency_Proprietaire' which I removed for log-reg. I am not worried about variables being lineraly dependent when using trees.

In [ ]:
dfc = pd.concat([dfn_sc, dfcat], axis=1)
X = dfc
y_names = 'Credit_card', 'Savings', 'Morgage'
y = df[y_names[1]]
y.value_counts()
y[y==0].size/y.size
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)
print(f'Accuracy: {accuracy_score(y_test, y_pred)*100:.2f}%')

## Predicion for 'Savings'

In [ ]:
#%% ROC
fpr, tpr, thres = roc_curve(y_test, y_pred_proba[:,1])

df_roc = pd.DataFrame(dict(fpr=fpr, tpr=tpr, fpr_=fpr))
ax = df_roc.plot.line(x='fpr', y='tpr', title=f'auc={auc(fpr, tpr):.4f}')
df_roc.plot.line(x='fpr', y='fpr_', ax=ax, grid=True, style='--')
ax.set_xlabel("False positive rate")
ax.set_ylabel("True positive rate")
ax.get_legend().remove()

# xgboost classifier
## Predicion for 'Savings'

In [ ]:
import xgboost as xgb
from sklearn.metrics import accuracy_score

dtrain = xgb.DMatrix(X_train, label=y_train)
dtest =  xgb.DMatrix(X_test, label=y_test)
evallist = [(dtest, 'eval'), (dtrain, 'train')]
num_round = 100
param = {'max_depth': 6, 'num_class': 2, 'eta': 0.3, 'objective': 'multi:softprob' }

bst = xgb.train(param, dtrain, num_round)
y_pred_prob = bst.predict(dtest)
y_pred = np.asarray([np.argmax(n) for n in y_pred_prob])
print(f'Accuracy: {accuracy_score(y_test, y_pred)*100:.2f}%')

In [ ]:
#%% ROC
fpr, tpr, thres = roc_curve(y_test, y_pred_proba[:,1])

df_roc = pd.DataFrame(dict(fpr=fpr, tpr=tpr, fpr_=fpr))
ax = df_roc.plot.line(x='fpr', y='tpr', title=f'auc={auc(fpr, tpr):.4f}')
df_roc.plot.line(x='fpr', y='fpr_', ax=ax, grid=True, style='--')
ax.set_xlabel("False positive rate")
ax.set_ylabel("True positive rate")
ax.get_legend().remove()

In [ ]:
vi = bst.get_score(importance_type='gain')
dfg = pd.DataFrame(vi.items(), columns = ('feature', 'importance'))
dfg.sort_values('importance', ascending=False).reset_index(drop=True)

# Summary

Variables 'Credit_card', 'Morgage' are easier to predict. Variable 'Savings' is more difficult. I am disappointed by random forest classifiers (both sklearn and xgboost implementations). And at the same time impressed by simple sklearn Logistic Regression.

I have not used variables 'date_issued_bank' and 'date_birth'.

Variable  TOP_SALARIE is most important predictor. But it does not look right to me. It is binary 0/1 variable. What is it? Is it real/artificial? How is it obtained?

### todo:
- equalize 0s and 1s in input set (generate fake records)
- are we allowed to use 'Credit_card', 'Morgage' as input to predicting 'Savings'?
- check covariance 'Credit_card', 'Morgage' and 'Savings'

### questions:
- how to assess usefulness (commercial value) of a model?
- is there a known price of fp and fn? They are probably different. How costly is failed sale attempt? How costly is lost customer we failed make sale to?

### things I tried with minor or no effect:
- add higher order values of features (REVENU^2, NB_child^2, age^2)
- use SVM with rbf kernel (training takes long, about 5 minutes)


## Call 1 summary
2021-01-21

- remove duplicates
- manually remove observations with salary below 1000euro
- manually restric to just ages 25-55
- add variable seniority: date_issued_bank - today() (in years)
- change all variables to categorical using quartiles